# Target selection for the DESI Peculiar Velocity survey
#### **_Authors_**: Christoph Saulder, Kelly Douglass, Cullan Howlett, Khaled Said

**NOTE 7th May 2021, Christoph noticed that the sweeps and derived files below duplicate SGA objects in the overlap regions between North and South imaging surveys. I have corrected this. Following some more testing, and given the opportunity to update the files one last time, we also modifed one of the FP colour cuts (filter_colour3) by +0.05 mags.**

**NOTE: Updated May 2021, prepared for Main Survey!! Fixed bug in Other SGA selection as this was erroneously removing some galaxies. Following low redshift success rates for pointings at larger radii, set TF objects to only have 2 axial pointings at +/-0.4R(26). Added more frequent pointings for extended galaxies (0.2, 0.4, 0.6, 0.8, 1.0), rather than (0.33, 0.67, 1.0) to increase amount of useful data, but while still providing low priority targeting solutions. Finally, added SGA_ID to final files.**

**NOTE: Updated April 2021, code was applying photo-z and magnitude cuts to SGA galaxies, which is undesirable. Corrected this.**

**NOTE: Updated March 2021, corrected bug in semi-major axis pointings, switched to D(26) for SGA, split into different priority lists, included all SGA objects that aren't FP/TF at low priority. Included hand-picked off-axis targets for very large gals. Duplicated targets in bright and dark time, but note that this is only for SV and will likely not be true later, with split based on SV data**

**NOTE: Updated 18th February 2021, changed to DR9, updated SGA selection and re-arranged the entire selection procedure, moved from composite models to Sersic (because DR9)**

**NOTE: Updated 6th January 2021, to correct for a sign error in the position angle which causes TF and EXT object pointings to be misaligned relative to galaxy imaging, and to use sersic_R (radius) and SGA_pa (position angle) from more up-to-date SGA data**

This file contains code and description for defining the DESI PV targets. The final filenames correspond to 
* **BRIGHT_LOW** - Bright time targets only. Lowest priority. Only intended to provide targets for fibres that would otherwise go to sky.
* **BRIGHT_MEDIUM** - Bright time targets only. Medium priority. Only intended to provide targets for fibres that would otherwise go to sky, but more useful for us than LOW.
* **BRIGHT_HIGH** - Bright time targets only. Highest priority.
* **DARK_LOW** - Dark time targets only. Lowest priority. Only intended to provide targets for fibres that would otherwise go to sky.
* **DARK_MEDIUM** - Dark time targets only. Medium priority. Only intended to provide targets for fibres that would otherwise go to sky, but more useful for us than LOW.
* **DARK_HIGH** - Dark time targets only. Highest priority.

For SV purposes, we are duplicating our targets in both bright and dark time, however based on the SV data for later observations we will likely place subsets of galaxies only in one or the other. Hence we are providing 6 separate files to match what we will hopefully be requesting during main survey later.

__If you are only interested in the differences between final files, you can stop here.__ 

For our science purposes, the above files contain 5 target types.
* **FPT** - Fundamental Plane Target. Target placed on centre. High priority
* **TFT** - Tully-Fisher Target. Targets placed along semi-major axis at high priority, and on centre at medium priority.
* **SGA** - SGA galaxy that is smaller than the DESI fibre patrol radius, but may provide so provide targeting solutions where no other galaxy science objects are available. Target placed on centre at low priority.
* **EXT** - Extended objects that are bigger than DESI fibre patrol radius and so provide targeting solutions where no other galaxy science objects are available. Targets placed along semi-major axis at low priority.
* **EOA** - Extended objects that are bigger than DESI fibre patrol radius and so provide targeting solutions where no other galaxy science objects are available. Targets hand-picked off-axis at low priority.
These are defined first, before then being split over the final six target lists: bright/dark and low/medium/high priority. 

Note that these target lists are larger than the nominal target density listed in the proposal. This is because during SV we are requesting that Tully-Fisher galaxies are split into more targets (6 per galaxy + 1 in centre) than during main survey (2 per galaxy) and are observed in bright and dark time. This will enable us to test the extent to which the DESI instrument would be able to observe these galaxies and how far along their semi-major axis. During main survey these would only be observed 2 targets per galaxy (+ centre at lower priority).

The first code segment just runs through all the DR8 sweeps in the north and south, and the SGA catalogue, to identify objects of interest and get properties we will need to define the three samples above. The following cuts are applied:
* Z_PHOT_MEDIAN < 0.15 and Z_PHOT_L95 < 0.1. This selects nearby objects that are useful for peculiar velocity surveys. Typical scatter between SGA spec-z and photo-z is 0.02, so these cuts ensure we have as close to complete sample at z<0.1 as possible using photo-z only. See function `redshift_filter`
* The object must be at least one of:
    * The BGS target selection including bitmasks, colour cuts, and r-band fibre_mag cuts. See function `BGS_filter`
    * Or, in the BGS bitmask with magnitude limit r<18.0 and TYPE != PSF. See function `BGS_filter`
    * Or, cross-matched to the SGA catalogue within 1". See function `SGA_filter`

All objects within this selection are output as seperate north and south files from which we can then define the various sub-samples. These sub-samples are tackled in later sections of the notebook because this first stage takes a while (something like 4 hours), and so we have set it up to only (ideally) be run once.

In [1]:
import glob
import numpy as np
import scipy as sp
import astropy.units as u
from astropy.io import fits
from astropy.table import Table, Column
from astropy.coordinates import SkyCoord

In [3]:
#all definition for the basic selection


# Routine to convert from flux to magnitude
def get_mags(flux, fibre_flux, transmission, flux_ivar):
    
    mag = 22.5 - 2.5*np.log10(flux/transmission)
    mag_err = np.abs(2.5/(np.log(10)*flux/transmission*np.sqrt(flux_ivar)))
    fibre_mag = 22.5 - 2.5*np.log10(fibre_flux/transmission)
    
    return mag, mag_err, fibre_mag


# A function to compute radii and other shape information for the galaxies in our first selection
def get_shapes(shape_r,shape_e1,shape_e2,shape_r_ivar,shape_e1_ivar,shape_e2_ivar):
    
    rad_uncor = shape_r
    help_epsilon = np.sqrt(shape_e1**2 + shape_e2**2)
    BA_ratio = (1.0 - help_epsilon) / (1.0 + help_epsilon)
    rad_circ = rad_uncor * np.sqrt(BA_ratio)
    pos_angle = 0.5 * np.arctan2(shape_e2, shape_e1) / np.pi * 180.0
    err_shape_r = 1./np.sqrt(shape_r_ivar)
    err_shape_e1 = 1./np.sqrt(shape_e1_ivar)
    err_shape_e2 = 1./np.sqrt(shape_e2_ivar)
    return rad_uncor,rad_circ,BA_ratio,pos_angle,err_shape_r,err_shape_e1,err_shape_e2


#get all parameters from SGA file directly, except redshift
def get_SGA_measurements(SGA_dataset_ellipse, SGA_dataset_tractor):

    SGA_collect=np.zeros(len(SGA_dataset_ellipse),dtype=SGA_extra_datatype)
    
    #general data. Use TRACTOR RA/DEC rather than LEDA.
    SGA_collect['RA']=SGA_dataset_tractor['RA']
    SGA_collect['DEC']=SGA_dataset_tractor['DEC']
    
    #SGA specfic data
    SGA_collect['SGA_ID']=SGA_dataset_ellipse['SGA_ID']
    SGA_collect['mag_B'] = SGA_dataset_ellipse['MAG_LEDA'] 
    SGA_collect['size_SGA'] = SGA_dataset_ellipse['GROUP_DIAMETER'] 
    SGA_collect['SGA_redshift'] = SGA_dataset_ellipse['Z_LEDA'] 
    SGA_collect['SGA_MORPHTYPE'] = SGA_dataset_ellipse['MORPHTYPE'] 
    SGA_collect['SGA_pa']  = SGA_dataset_ellipse['PA']    
    SGA_collect['SGA_ba']  = SGA_dataset_ellipse['BA']  
    SGA_collect['SB_D25_g']  = SGA_dataset_ellipse['G_MAG_SB25'] 
    SGA_collect['SB_D25_r']  = SGA_dataset_ellipse['R_MAG_SB25'] 
    SGA_collect['SB_D25_z'] = SGA_dataset_ellipse['Z_MAG_SB25'] 
    SGA_collect['RADIUS_SB25']  = SGA_dataset_ellipse['SB_D25_LEDA']  
    
    return SGA_collect


# A function defining the BGS or other bright objects filter
def BGS_filter(sweepdata, mag_g, mag_r, mag_z, fibremag_r, basic_cuts, redshiftmask, skymask):
    
    # BGS selection    
    help_LS_rr = 22.5 - 2.5*np.log10(sweepdata['FLUX_R'])
    filter_gaia_bright = (sweepdata['GAIA_PHOT_G_MEAN_MAG'] == 0)
    filter_gaia = ((sweepdata['GAIA_PHOT_G_MEAN_MAG'] - help_LS_rr) > 0.6) | filter_gaia_bright
    filter_bgsmask = (sweepdata['MASKBITS'] & (2**1) == 0) & (sweepdata['MASKBITS'] & (2**12) ==0) & (sweepdata['MASKBITS'] & (2**13) == 0)
    filter_fracmasked = (sweepdata['FRACMASKED_G'] < 0.4) & (sweepdata['FRACMASKED_R'] < 0.4) & (sweepdata['FRACMASKED_Z'] < 0.4)
    filter_fracflux = (sweepdata['FRACFLUX_G'] < 5.0) & (sweepdata['FRACFLUX_R'] < 5.0) & (sweepdata['FRACFLUX_Z'] < 5.0)
    filter_fracin = (sweepdata['FRACIN_G'] > 0.3) & (sweepdata['FRACIN_R'] > 0.3) & (sweepdata['FRACIN_Z'] > 0.3)
    
    gmr, rmz = mag_g - mag_r, mag_r - mag_z
    bgs_magcut = (-1 < gmr) & (gmr < 4) & (-1 < rmz) & (rmz < 4)
    bgs_fibremag_cut = ((fibremag_r < (5.1 + mag_r)) & (mag_r <= 17.8)) | ((fibremag_r < 22.9) & (mag_r > 17.8) & (mag_r < 20))

    allbgsmask = basic_cuts & bgs_magcut & bgs_fibremag_cut & filter_gaia & filter_fracmasked & filter_bgsmask & filter_fracflux & filter_fracin & redshiftmask & skymask

    # Other bright things selection
    filter_bright_extended = (mag_r < 18.0) & (sweepdata['TYPE'] != 'PSF ')
    allbrightmask = basic_cuts & filter_bright_extended & filter_gaia_bright & filter_bgsmask & filter_fracmasked & redshiftmask & skymask
    
    return allbgsmask, allbrightmask


# A function defining the SGA filter
def SGA_filter(sweepdata, coord_sweepdata, mag_g, mag_r, SGA_coords, skymask):

    #now that SGA and the other photometric data are both DR9, they sould match perfectly ... so only rounding error tolerance (tests with topcat confirm it)
    delta_degree=(0.01/3600.)
        
    # Removes spurious objects close to the SGA centres that are not SGA galaxies to make the cross-matching faster and applies
    # the North/South imaging cut
    onlybrightenough=(mag_g < 20.0) & (mag_r < 20.0) & (sweepdata['TYPE'] != b'PSF ') & skymask

    nearestneighbor_SGA, d2d, _ = coord_sweepdata[onlybrightenough].match_to_catalog_sky(SGA_coords)  
    SGA_overlap = (d2d < (delta_degree*u.degree))
                
    allsgamask = onlybrightenough
    allsgamask[onlybrightenough] = SGA_overlap
        
    SGA_overlap_indices = np.zeros(len(onlybrightenough), dtype=np.int64)-1
    SGA_overlap_indices[np.where(allsgamask == True)] = nearestneighbor_SGA[SGA_overlap]
    
    SGA_overlap_indices = SGA_overlap_indices[(SGA_overlap_indices>-1)]
    
    return allsgamask, SGA_overlap_indices


# Defines our redshift cuts
def redshift_filter(sweepdata):
    
    # filter high redshift galaxies           
    return (sweepdata['Z_PHOT_MEDIAN'] <= 0.15) & (sweepdata['Z_PHOT_L95'] <= 0.1)
    
    
# Defines a filter for the south
def south_filter(coord_sweepdata):
    
    sc_gal = coord_sweepdata.galactic      
    footprint_south = (sc_gal.b/u.degree <= 0.0) | (coord_sweepdata.dec/u.degree <= 32.375)
    
    return footprint_south


# Defines a filter for the north
def north_filter(coord_sweepdata):
    
    sc_gal = coord_sweepdata.galactic 
    footprint_north = (sc_gal.b/u.degree > 0.0) & (coord_sweepdata.dec/u.degree > 32.375)
    
    return footprint_north


# A function to run over all the sweeps and return the data we care about
def get_sweeps(fname, fname_photoz, SGA_collect, area='north'):
    sweepdata = fits.open(fname,memmap=True)[1].data
    photozsweepdata = fits.open(fname_photoz,memmap=True)[1].data
    
    SGA_coords = SkyCoord(ra = SGA_collect['RA']*u.degree, dec = SGA_collect['DEC']*u.degree) 

    # Compute the magnitudes
    mag_g, mag_g_err, fibre_mag_g = get_mags(sweepdata['FLUX_G'], sweepdata['FIBERFLUX_G'], sweepdata['MW_TRANSMISSION_G'], sweepdata['FLUX_IVAR_G'])
    mag_r, mag_r_err, fibre_mag_r = get_mags(sweepdata['FLUX_R'], sweepdata['FIBERFLUX_R'], sweepdata['MW_TRANSMISSION_R'], sweepdata['FLUX_IVAR_R'])
    mag_z, mag_z_err, fibre_mag_z = get_mags(sweepdata['FLUX_Z'], sweepdata['FIBERFLUX_Z'], sweepdata['MW_TRANSMISSION_Z'], sweepdata['FLUX_IVAR_Z'])
    
    # Get the north/south sky cuts
    coord_sweepdata = SkyCoord(ra = sweepdata['RA']*u.degree, dec = sweepdata['DEC']*u.degree) 
    if area.lower() == 'north':
        skymask = north_filter(coord_sweepdata)
    else:
        skymask = south_filter(coord_sweepdata)
    
    # Get the SGA filter
    allsgamask, SGA_overlap_indices = SGA_filter(sweepdata, coord_sweepdata, mag_g, mag_r, SGA_coords, skymask)

    # Some basic filters to remove fluff
    obsfilter = (sweepdata['NOBS_G'] > 0) & (sweepdata['NOBS_R'] > 0) & (sweepdata['NOBS_Z'] > 0)
    filter_minflux = (sweepdata['FLUX_G'] > 0) & (sweepdata['FLUX_R'] > 0) & (sweepdata['FLUX_Z'] > 0)
    basic_cuts = obsfilter &  filter_minflux

    #nbasic = np.sum(basic_cuts)
    #sweepdata = sweepdata[basic_cuts]
    #photozsweepdata = photozsweepdata[basic_cuts]
    
    #coord_sweepdata = SkyCoord(ra = sweepdata['RA']*u.degree, dec = sweepdata['DEC']*u.degree) 

    #sky_red_filter = redshiftmask & skymask
    
    #sweepdata = sweepdata[sky_red_filter]
    #photozsweepdata = photozsweepdata[sky_red_filter]    

    # Get the redshift filter
    redshiftmask = redshift_filter(photozsweepdata)
    
    # Compute some shape parameters
    rad_uncor,rad_circ,BA_ratio,pos_angle,err_shape_r,err_shape_e1,err_shape_e2=get_shapes(sweepdata['SHAPE_R'],sweepdata['SHAPE_E1'],sweepdata['SHAPE_E2'],sweepdata['shape_r_ivar'],sweepdata['shape_e1_ivar'],sweepdata['shape_e2_ivar'])

    # Get the BGS and bright things filter with a photo-z cut
    allbgsmask, allbrightmask = BGS_filter(sweepdata, mag_g, mag_r, mag_z, fibre_mag_r, basic_cuts, redshiftmask, skymask)

    # Finally, pull everything together and return objects we want
    full_filter = (allbrightmask | allsgamask | allbgsmask)
    
    nfull = np.sum(full_filter)
    allgal = np.zeros(nfull, dtype = galaxydatatype)

    #transfer general properties from sweep
    for name in ["OBJID", "BRICKID", "BRICKNAME", "RA", "DEC", "TYPE", "SERSIC"]:
        allgal[name] = sweepdata[name][full_filter]

    #get photometric redshifts
    for name in ["Z_PHOT_MEDIAN", "Z_PHOT_L95"]:
        allgal[name] = photozsweepdata[name][full_filter]
    
    #transfer precomputed parameters
    names = ["mag_g", "mag_r", "mag_z", "mag_g_err", "mag_r_err", "mag_z_err", "fibre_mag_g", "fibre_mag_r", "fibre_mag_z", "uncor_radius","circ_radius","BA_ratio","pos_angle","err_shape_r","err_shape_e1","err_shape_e2"]
    for name, vals in zip(names, [mag_g, mag_r, mag_z, mag_g_err, mag_r_err, mag_z_err, fibre_mag_g, fibre_mag_r, fibre_mag_z,rad_uncor,rad_circ,BA_ratio,pos_angle,err_shape_r,err_shape_e1,err_shape_e2]):
        allgal[name] = vals[full_filter]
        
    #set flags
    allgal['inSGA'] = np.where(allsgamask[full_filter], 1, 0)
    allgal['inBGS'] = np.where(allbgsmask[full_filter], 1, 0)
    allgal['inlocalbright'] = np.where(allbrightmask[full_filter], 1, 0)  
    
    insga=(allgal['inSGA']==1)
    
    # Add SGA info where available. Objects without a cross match have inSGA == 0, SGA_ID = -1, and other properties left empty.
    allgal['SGA_ID'] = -1
    for na in SGA_collect.dtype.names:
         allgal[na][insga]=SGA_collect[na][SGA_overlap_indices]
     
    return allgal    


In [4]:
#run the basic selection

#calculate everything for SGA    
SGA_collect=get_SGA_measurements(SGA_dataset_ellipse, SGA_dataset_tractor)

# This takes a while (4 hours or so), so by default it assumes we have already generated these files and does nothing. 
# If you need to run them again, just set 'if True'.
if True:
    # First do South
    fullsweep = []
    for i, fname in enumerate(fnamesouth):
        #path to rongpu's photometric redshifts ... not officially released yet
        #fname_photoz='/global/cscratch1/sd/rongpu/dr9_photoz/south/'+fname[-26:-5]+'-pz.fits'
        #the files will be moved to the following directory in the near future
        fname_photoz=pathsouth+'9.0-photo-z/'+fname[-26:-5]+'-pz.fits'
        allgal = get_sweeps(fname, fname_photoz, SGA_collect, area="south")
        print(i, len(fnamesouth), len(allgal))
        fullsweep.append(allgal)    

    fullsweep_south = np.concatenate(fullsweep,axis=0)
    np.save(savedatapath+"fullsweep_south", fullsweep_south)       

    # Then do North
    fullsweep = []
    for i, fname in enumerate(fnamenorth):
        #path to rongpu's photometric redshifts ... not officially released yet
        #fname_photoz='/global/cscratch1/sd/rongpu/dr9_photoz/north/'+fname[-26:-5]+'-pz.fits'
        #the files will be moved to the following directory in the near future
        fname_photoz=pathnorth+'9.0-photo-z/'+fname[-26:-5]+'-pz.fits'
        allgal = get_sweeps(fname, fname_photoz, SGA_collect, area="north")
        print(i, len(fnamenorth), len(allgal))
        fullsweep.append(allgal)    

    fullsweep_north = np.concatenate(fullsweep,axis=0)
    np.save(savedatapath+"fullsweep_north", fullsweep_north)       

/tmp/ipykernel_43910/3076960452.py:7: RuntimeWarning: divide by zero encountered in log10
  mag = 22.5 - 2.5*np.log10(flux/transmission)
/tmp/ipykernel_43910/3076960452.py:7: RuntimeWarning: invalid value encountered in log10
  mag = 22.5 - 2.5*np.log10(flux/transmission)
/tmp/ipykernel_43910/3076960452.py:8: RuntimeWarning: divide by zero encountered in true_divide
  mag_err = np.abs(2.5/(np.log(10)*flux/transmission*np.sqrt(flux_ivar)))
/tmp/ipykernel_43910/3076960452.py:9: RuntimeWarning: divide by zero encountered in log10
  fibre_mag = 22.5 - 2.5*np.log10(fibre_flux/transmission)
/tmp/ipykernel_43910/3076960452.py:22: RuntimeWarning: divide by zero encountered in true_divide
  err_shape_r = 1./np.sqrt(shape_r_ivar)
/tmp/ipykernel_43910/3076960452.py:23: RuntimeWarning: divide by zero encountered in true_divide
  err_shape_e1 = 1./np.sqrt(shape_e1_ivar)
/tmp/ipykernel_43910/3076960452.py:24: RuntimeWarning: divide by zero encountered in true_divide
  err_shape_e2 = 1./np.sqrt(shape

0 455 5618


/tmp/ipykernel_43910/3076960452.py:7: RuntimeWarning: divide by zero encountered in log10
  mag = 22.5 - 2.5*np.log10(flux/transmission)
/tmp/ipykernel_43910/3076960452.py:7: RuntimeWarning: invalid value encountered in log10
  mag = 22.5 - 2.5*np.log10(flux/transmission)
/tmp/ipykernel_43910/3076960452.py:8: RuntimeWarning: divide by zero encountered in true_divide
  mag_err = np.abs(2.5/(np.log(10)*flux/transmission*np.sqrt(flux_ivar)))
/tmp/ipykernel_43910/3076960452.py:9: RuntimeWarning: divide by zero encountered in log10
  fibre_mag = 22.5 - 2.5*np.log10(fibre_flux/transmission)


1 455 5291
2 455 13396
3 455 9577
4 455 12830
5 455 5289
6 455 14326
7 455 6392
8 455 667
9 455 2200
10 455 241
11 455 11326
12 455 9093
13 455 77
14 455 34
15 455 8251
16 455 11308
17 455 12593
18 455 473
19 455 5165
20 455 3816
21 455 8333
22 455 7684
23 455 8663
24 455 14796
25 455 2593
26 455 4909
27 455 2479
28 455 12553
29 455 13743
30 455 13185
31 455 12425
32 455 4575
33 455 6372
34 455 11170
35 455 10590
36 455 11265
37 455 18970
38 455 6785
39 455 6690
40 455 11858
41 455 12439
42 455 6750
43 455 10075
44 455 0
45 455 0
46 455 4827
47 455 15883
48 455 34
49 455 985
50 455 0
51 455 6869
52 455 5657
53 455 15795
54 455 13365
55 455 4469
56 455 13578
57 455 7153
58 455 11592
59 455 11399
60 455 608
61 455 14363
62 455 7534
63 455 11258
64 455 3091
65 455 8257
66 455 11518
67 455 6966
68 455 14609
69 455 8904
70 455 4338
71 455 13139
72 455 15553
73 455 11475
74 455 4903
75 455 7591
76 455 15585
77 455 13145
78 455 9420
79 455 11796
80 455 10840
81 455 11361
82 455 15412
83 455 3

## PV_FP - Fundamental Plane Sample

Our FP selection is based on photometrically identifying likely elliptical galaxies using colour cuts and shape parameters. The colour cuts were tuned based on comparisons with visual morphologies from both the SGA and GalaxyZoo. We then simulated the S/N for each target using the fibre magnitude and worked out the magnitude limit above which we were able to recover a high fraction of S/N $> 7.5A^{-1}$ after at most 5x1000s dark time exposures. This S/N limit was chosen based on SV0 data as the point above which we could measure velocity dispersions with less than 10\% relative error. These considerations were then used to define the magnitude limit of the selection, so as to avoid wasting time on targets that would be unlikely to yield good enough S/N.

The selection function for the FP sample is then:
* (TYPE == DEV) OR (TYPE == SER AND (SERSIC > 2.5)) AND (circular radius > 0) AND (ellipticity < 0.7). This keeps elliptical shaped objects well fit by a De Vaucouleurs profile. See function `FP_shape_filter`
* And, colour cuts (g-r > 0.68) AND (g-r > 1.3[r-z]-0.05) AND (g-r < 2.0[r-z]-0.2). This removes objects in the blue cloud/green valley, dusty spirals, and mergers/peculiar objects. See function `FP_colour_filter`
* And, magnitude cut r < 18.0. Beyond this there are generally fewer targets due to our redshift cut, and the velocity dispersion success rate per 0.1 mag bin drops below 50% for a single exposure, and 97.5% after a maximum of 5x1000s dark time exposures. So this is a reasonable sweet spot between capitalising on DESI's abilities (it's a full mag deeper than the SDSS PV survey), but not wasting time on targets or repeating excessively. See function `FP_mag_filter`

We then restrict our selection to objects with Dec > -30.0. See function `DESI_footprint_filter`.

In [5]:
def FP_shape_filter(fullsweep_combi,fp_sersic_limit):

    # Shape filters to check for ellipticals
    filter_rad = (fullsweep_combi['circ_radius'] > 0)
    filter_ellipticity = ((1.0 - fullsweep_combi['BA_ratio']) < 0.7)
    filter_morph = (((fullsweep_combi['TYPE'] == b'DEV ') | (fullsweep_combi['TYPE'] == b'SER')) & (fullsweep_combi['SERSIC'] > fp_sersic_limit))

    return filter_morph & filter_ellipticity & filter_rad

def FP_colour_filter(fullsweep_combi):
    
    #colour cuts to remove blue cloud/green valley, dusty spirals and mergers/peculiar objects
    gmr = fullsweep_combi['mag_g'] - fullsweep_combi['mag_r']
    rmz = fullsweep_combi['mag_r'] - fullsweep_combi['mag_z']
    filter_colour1 = (gmr > 0.68)
    filter_colour2 = (gmr > 1.30*rmz - 0.05)
    filter_colour3 = (gmr < 2.00*rmz - 0.15)
    
    return filter_colour1 & filter_colour2 & filter_colour3
    
def FP_mag_filter(fullsweep_combi,fp_rband_limit):
    
    # Magnitude limit beyond which the simulated velocity dispersion success rate is too low
    return (fullsweep_combi['mag_r'] <= fp_rband_limit)

def DESI_footprint_filter(fullsweep_combi): #, desi_tiles
    
    # Cross match the object coordinates with the DESI tiles
    #coord_fullsweep_combi = SkyCoord(ra = fullsweep_combi['RA']*u.degree, dec = fullsweep_combi['DEC']*u.degree) 
    #coord_DESI_tilepointing = SkyCoord(ra = desi_tiles['RA']*u.degree, dec = desi_tiles['DEC']*u.degree) 
    #desi_tile_rad = 1.628 * u.degree

    #nearestneighbor, d2d, _ = coord_fullsweep_combi.match_to_catalog_sky(coord_DESI_tilepointing)  
    #survey_overlap = (d2d < desi_tile_rad)
    
    #everything with grz photometric data north of -30 degree ... based on David's comment
    survey_overlap=(fullsweep_combi['DEC']>-30.0)

    return survey_overlap

def inSGA_filter(fullsweep_combi):
    
    # Only objects that are in the SGA.
    return (fullsweep_combi["inSGA"] == 1)

# Sometimes the SGA matching finds multiple profiles at the same coordinates ... but we just need each object once
# this function became obsolete once all input was from the same source (DR9) -CS
#def cleanmulti(fullsweep_combi):
    
#    inSGA = inSGA_filter(fullsweep_combi)
    
    # Identify objects for which the SGA_ID appears more than once
#    multi_sga = np.unique(fullsweep_combi['SGA_ID'][inSGA],return_counts=True)[1]
#    multi_sgamatch = (multi_sga > 1)
    
#    rejectgalaxies = np.zeros(len(fullsweep_combi),dtype=bool)
#    problemsgaidlist = np.unique(fullsweep_combi['SGA_ID'][inSGA])[multi_sgamatch]

    #mark galaxies for rejection
#    counter = 0
#    for multi_id in problemsgaidlist:
#        if counter % 100 == 0:
#            print(counter, len(problemsgaidlist))
#        currentid = (multi_id == fullsweep_combi['SGA_ID'])
#        keepgal = np.argmin(fullsweep_combi['mag_r'][currentid])
#        multis = np.ones(np.sum(currentid),dtype=bool)
#        multis[keepgal] = 0
#        (rejectgalaxies[currentid]) = multis
#        counter += 1
    
    #for the filter, we want to select the galaxies to keep
#    keepgalaxies = np.invert(rejectgalaxies)    
#    return keepgalaxies

In [6]:
# Load the DESI tile centres. These are somewhere in NERSC, but not sure where. 
# So grabbed from the wiki. Should update once we work out where they are for posterity.
desi_alltiles = fits.open('desi-tiles.fits', memmap=True)[1].data
desi_tiles = desi_alltiles[desi_alltiles['IN_DESI']==1]

# Read in the north and south properties and concatenate them
fullsweep_north = np.load(savedatapath+'fullsweep_north.npy')
fullsweep_south = np.load(savedatapath+'fullsweep_south.npy')
fullsweep_combi = np.concatenate([fullsweep_north, fullsweep_south],axis=0)

# Only keep objects in the (ever changing) DESI spectroscopic footprint
footprint_filter = DESI_footprint_filter(fullsweep_combi)#, desi_tiles)
#low_red_filter = lower_red_filter(fullsweep_combi)
index = np.where(footprint_filter)
fullsweep_combi = fullsweep_combi[index]

#clean data by removing multiple SGA matches ... obsolete with DR9 - CS
#index = cleanmulti(fullsweep_combi)
#fullsweep_combi = fullsweep_combi[index]

# Compute all the FP filters 
mag_filter = FP_mag_filter(fullsweep_combi,fp_rband_limit)
shape_filter = FP_shape_filter(fullsweep_combi,fp_sersic_limit)
colour_filter = FP_colour_filter(fullsweep_combi)
index = np.where(mag_filter & shape_filter & colour_filter)

# Output the full target list properties for our benefit
filtered_data = fullsweep_combi[index]
ntargets = len(filtered_data)
print(ntargets, len(filtered_data[filtered_data['inSGA'] == 1]))
fits.writeto(savedatapath+"pv_fp_full.fits", filtered_data, overwrite=True)

# Output the reduced target list as required, plus some other information we might need for our purposes
target_data = Table([Column(filtered_data["OBJID"], name='OBJID'),
                     Column(filtered_data["BRICKID"], name='BRICKID'),
                     Column(filtered_data["BRICKNAME"], name='BRICKNAME'),
                     Column(filtered_data["RA"], name='RA'), 
                     Column(filtered_data["DEC"], name='DEC'), 
                     Column(np.zeros(ntargets, dtype='>f4'), name='PMRA'), 
                     Column(np.zeros(ntargets, dtype='>f4'), name='PMDEC'), 
                     Column(np.full(ntargets, 2015.5, dtype='>f4'), name='REF_EPOCH'), 
                     Column(np.full(ntargets, False, dtype='bool'), name='OVERRIDE'), 
                     Column(np.full(ntargets, "FPT", dtype='>S3'), name='PVTYPE'), 
                     Column(np.full(ntargets, 1, dtype='>i4'), name='PVPRIORITY'),
                     Column(np.full(ntargets, 1, dtype='>i4'), name='POINTINGID'),
                     Column(filtered_data["SGA_ID"], name='SGA_ID')])
target_data.write(savedatapath+"pv_fp.fits", format='fits', overwrite=True)

427273 98687


## PV_TF - Tully-Fisher sample

Our TF targets are based on galaxies in SGA. We then apply the inverse of the FP filters, restricting our TF sample to objects with spiral shapes. The following routines are based on Kelly Douglass's code [here](https://github.com/kadglass/DESI_SGA/blob/master/sga_semimajor_ends.ipynb)

The selection function for the TF sample is then:
* inSGA == 1. This means the object is in the SGA catalogue. See function `inSGA_filter` (in previous code block so needs caching)
* (TYPE == EXP)  AND (circular radius > 0) AND (b/a < cos(25.0)). This keeps highly inclined objects well fit by an exponential profile. See function `TF_shape_filter`

We then restrict our selection to objects with Dec >= -30.0. See function `DESI_footprint_filter`.

Out targets for the TF sample are the points at the outer edges of the disk, and the centre at lower priority. The centres of all these targets are expected to already by included in the BGS, and so are put in with OVERRIDE == FALSE to capture this possible overlap. In order to generate our targets, we want to perturb the fibre positions from the centre along the position angle as close to either edge of the disk as possible. These two perturbed pointings are then added to the target list, along with OVERRIDE == TRUE to ensure the fibre is actually placed at the disk edge. See function `TF_disk_edges`. 

However, the exact multiple of R(26) to use is still unknown at the moment and depends on the Halpha flux we can actually get redshift for using redrock. This is currently unclear and so during SV, we are in fact providing targets spaced along the semi-major axis (the same as for our lower priority EXT targets below) at 0.33, 0.67 and 1.0 R(26). See function `grid_positions`. This means the total number of TF targets is larger than the nominal number we asked for during main survey operations, but they can be subsampled down to our requested target density as specified in the DESI Secondary Target proposal description.

**Note: Updated for main survey following SV3 data. Set fibre positions to +/-0.4R(26).**

In [7]:
def TF_shape_filter(fullsweep_combi,tf_ratiomin,tf_sersic_limit):

    # Shape filters to check for inclined spirals
    filter_rad = (fullsweep_combi['uncor_radius'] > 0)
    filter_ellipticity = (fullsweep_combi['BA_ratio'] <= tf_ratiomin)
    filter_morph = (fullsweep_combi['TYPE'] == b'EXP ')| ((fullsweep_combi['TYPE'] == b'SER') & (fullsweep_combi['SERSIC'] < tf_sersic_limit))
    #added sersic criterium, because SER is now(DR9) triggered very often -CS
        
    filter_size = (fullsweep_combi["size_SGA"] >= 20.0/60.0)   # There are a few objects that have DIAM < 20" still in SGA so lets remove these for ease.
        
    return filter_morph & filter_ellipticity & filter_rad & filter_size

def TF_disk_edges(filtered_data):
    
    r50 = filtered_data['size_SGA'] / 120.   # arcminutes -> degrees, diameter -> radius
    x = np.array([-0.4, 0.0, 0.4]).reshape((1,3))
    
    # Distances along the semi-major axis from the center coordinate for our targets
    delta_a = np.dot(r50.reshape((len(r50),1)),x).T
    
    # Target positions
    og_position = SkyCoord(filtered_data['RA']*u.deg, filtered_data['DEC']*u.deg, frame='icrs')
    fiber_pos = og_position.directional_offset_by(filtered_data['SGA_pa']*u.deg, delta_a*u.deg)
    fiber_ra, fiber_dec = fiber_pos.ra.value.T.flatten(), fiber_pos.dec.value.T.flatten()

    fiber_id = np.tile(np.arange(1,4),len(filtered_data))
    pvpriority = np.tile(np.array([1, 2, 1], dtype='>i4'), len(filtered_data))   # Edges at high prority, centre at low
    override = np.tile(np.array([True, False, True], dtype='bool'), len(filtered_data)) # Edges have override, but not centre.
    objid = np.repeat(filtered_data["OBJID"], 3)
    brickid = np.repeat(filtered_data["BRICKID"], 3)
    brickname = np.repeat(filtered_data["BRICKNAME"], 3)
    sga_id = np.repeat(filtered_data["SGA_ID"], 3)
    
    return fiber_ra, fiber_dec, fiber_id, objid, brickid, brickname, pvpriority, override, sga_id

def axis_positions(filtered_data):
    
    r50 = filtered_data['size_SGA'] / 120.   # arcminutes -> degrees, diameter -> radius
        
    #x = np.concatenate((np.linspace(-3,-0.5,6), np.linspace(0.5,3,6))).reshape((1,12))
    x = np.array([-1.0, -0.67, -0.33, 0.0, 0.33, 0.67, 1.0]).reshape((1,7))
    
    # Distances along the semi-major axis from the center coordinate for our targets
    delta_a = np.dot(r50.reshape((len(r50),1)),x).T
    
    # Target positions
    og_position = SkyCoord(filtered_data['RA']*u.deg, filtered_data['DEC']*u.deg, frame='icrs')
    fiber_pos = og_position.directional_offset_by(filtered_data['SGA_pa']*u.deg, delta_a*u.deg)
    fiber_ra, fiber_dec = fiber_pos.ra.value.T.flatten(), fiber_pos.dec.value.T.flatten()

    #fiber_id = np.tile(np.arange(1,13),len(filtered_data))
    #objid = np.repeat(filtered_data["OBJID"], 12)
    #brickid = np.repeat(filtered_data["BRICKID"], 12)
    #brickname = np.repeat(filtered_data["BRICKNAME"], 12)

    fiber_id = np.tile(np.arange(1,8),len(filtered_data))
    pvpriority = np.tile(np.array([1, 1, 1, 2, 1, 1, 1], dtype='>i4'), len(filtered_data))   # Edges at high prority, centre at low
    override = np.tile(np.array([True, True, True, False, True, True, True], dtype='bool'), len(filtered_data)) # Edges have override, but not centre.
    objid = np.repeat(filtered_data["OBJID"], 7)
    brickid = np.repeat(filtered_data["BRICKID"], 7)
    brickname = np.repeat(filtered_data["BRICKNAME"], 7)
    sga_id = np.repeat(filtered_data["SGA_ID"], 7)

    return fiber_ra, fiber_dec, fiber_id, objid, brickid, brickname, pvpriority, override, sga_id


In [8]:
# Load the DESI tile centres. These are somewhere in NERSC, but not sure where. 
# So grabbed from the wiki. Should update once we work out where they are for posterity.
desi_alltiles = fits.open('desi-tiles.fits', memmap=True)[1].data
desi_tiles = desi_alltiles[desi_alltiles['IN_DESI']==1]

# Read in the north and south properties and concatenate them
fullsweep_north = np.load(savedatapath+'fullsweep_north.npy')
fullsweep_south = np.load(savedatapath+'fullsweep_south.npy')
fullsweep_combi = np.concatenate([fullsweep_north, fullsweep_south],axis=0)

# Only keep objects in the DESI footprint and SGA
SGA_only_filter = inSGA_filter(fullsweep_combi)
footprint_filter = DESI_footprint_filter(fullsweep_combi)#, desi_tiles)
index = np.where(footprint_filter & SGA_only_filter)
fullsweep_combi = fullsweep_combi[index]

#clean data by removing multiple SGA matches ... obsolete with DR9, --CS
#index = cleanmulti(fullsweep_combi)
#fullsweep_combi = fullsweep_combi[index]

# Compute all the TF filters 
shape_filter = TF_shape_filter(fullsweep_combi,tf_ratiomin,tf_sersic_limit)
index = np.where(shape_filter)

# Output the underlying galaxies 
filtered_data = fullsweep_combi[index]
ngalaxies = len(filtered_data)
fits.writeto(savedatapath+"pv_tf_full.fits", filtered_data, overwrite=True)

# Get the target positions by perturbing along the semi-major axis
fiber_ra, fiber_dec, fiber_id, objid, brickid, brickname, pvpriority, override, sga_id = TF_disk_edges(filtered_data)
#fiber_ra, fiber_dec, fiber_id, objid, brickid, brickname, pvpriority, override, sga_id = axis_positions(filtered_data)  # For SV we are using a line of pointings along the semi-major axis
ntargets = len(fiber_ra)
print(ngalaxies, ntargets, len(filtered_data[filtered_data['inSGA'] == 1]))

# Output the targets
target_data = Table([Column(objid, name='OBJID'),
                     Column(brickid, name='BRICKID'),
                     Column(brickname, name='BRICKNAME'),
                     Column(fiber_ra, name='RA'), 
                     Column(fiber_dec, name='DEC'), 
                     Column(np.zeros(ntargets, dtype='>f4'), name='PMRA'), 
                     Column(np.zeros(ntargets, dtype='>f4'), name='PMDEC'), 
                     Column(np.full(ntargets, 2015.5, dtype='>f4'), name='REF_EPOCH'), 
                     Column(override, name='OVERRIDE'), 
                     Column(np.full(ntargets, "TFT", dtype='>S3'), name='PVTYPE'), 
                     Column(pvpriority, name='PVPRIORITY'),
                     Column(fiber_id, name='POINTINGID'),
                     Column(sga_id, name='SGA_ID')])
target_data.write(savedatapath+"pv_tf.fits", format='fits', overwrite=True)

130679 392037 130679


## PV_SGA - Other SGA objects sample

There are a number of relatively small SGA galaxies that don't make it into the FP or TF samples and aren't larger than a fibre patrol radius. However, these are still very easy to target with spare fibres and might be useful for providing targetting solutions during dark time (I think they will already be folded in to bright time). So we add them in just with a single pointing in the centre, with OVERRIDE == FALSE. We also put these at lowest priority, as they are not useful for PV science.

The selection function for the SGA sample is:
* inSGA == 1. This means the object is in the SGA catalogue. See function `inSGA_filter` (in previous code block so needs caching)
* Not FP or TF targets. See function `FP_shape_filter`, `mag_filter`, `colour_filter` and `TF_shape_filter`.
* Smaller than a DESI fibre patrol radius. See function `large_galaxy_filter`.

We then restrict our selection to objects with Dec >= -30.0. See function `DESI_footprint_filter`.

In [9]:
def large_galaxy_filter(fullsweep_combi):
    
    max_patrol_diam = 2.85 # arcminutes
    large_filter = (fullsweep_combi['size_SGA'] >= max_patrol_diam)

    return large_filter

In [10]:
# Load the DESI tile centres. These are somewhere in NERSC, but not sure where. 
# So grabbed from the wiki. Should update once we work out where they are for posterity.
desi_alltiles = fits.open('desi-tiles.fits', memmap=True)[1].data
desi_tiles = desi_alltiles[desi_alltiles['IN_DESI']==1]

# Read in the north and south properties and concatenate them
fullsweep_north = np.load(savedatapath+'fullsweep_north.npy')
fullsweep_south = np.load(savedatapath+'fullsweep_south.npy')
fullsweep_combi = np.concatenate([fullsweep_north, fullsweep_south],axis=0)

# Only keep objects in the DESI footprint and SGA
SGA_only_filter = inSGA_filter(fullsweep_combi)
footprint_filter = DESI_footprint_filter(fullsweep_combi)#, desi_tiles)
index = np.where(footprint_filter & SGA_only_filter)
fullsweep_combi = fullsweep_combi[index]

checkgal = np.where(fullsweep_combi["SGA_ID"] == 65)

# Clean data by removing multiple SGA matches  ..obsolete with DR9
#index = cleanmulti(fullsweep_combi)
#fullsweep_combi = fullsweep_combi[index]

# Keep only galaxies smaller than the patrol radius and which are not in the previous FP or TF selections
mag_filter = FP_mag_filter(fullsweep_combi,fp_rband_limit)
shape_filter_FP = FP_shape_filter(fullsweep_combi,fp_sersic_limit)
colour_filter = FP_colour_filter(fullsweep_combi)
full_filter_FP = mag_filter & shape_filter_FP & colour_filter
shape_filter_TF = TF_shape_filter(fullsweep_combi,tf_ratiomin,tf_sersic_limit)
large_filter = large_galaxy_filter(fullsweep_combi)
print(large_filter[checkgal], shape_filter_TF[checkgal], shape_filter_FP[checkgal])
index = np.where(~large_filter & ~shape_filter_TF & ~full_filter_FP)

# Output the underlying galaxies 
filtered_data = fullsweep_combi[index]
ntargets = len(filtered_data)
print(ntargets, len(filtered_data[filtered_data['inSGA'] == 1]))
fits.writeto(savedatapath+"pv_sga_full.fits", filtered_data, overwrite=True)
                              
# Output the reduced target list as required, plus some other information we might need for our purposes
target_data = Table([Column(filtered_data["OBJID"], name='OBJID'),
                     Column(filtered_data["BRICKID"], name='BRICKID'),
                     Column(filtered_data["BRICKNAME"], name='BRICKNAME'),
                     Column(filtered_data["RA"], name='RA'), 
                     Column(filtered_data["DEC"], name='DEC'), 
                     Column(np.zeros(ntargets, dtype='>f4'), name='PMRA'), 
                     Column(np.zeros(ntargets, dtype='>f4'), name='PMDEC'), 
                     Column(np.full(ntargets, 2015.5, dtype='>f4'), name='REF_EPOCH'), 
                     Column(np.full(ntargets, False, dtype='bool'), name='OVERRIDE'), 
                     Column(np.full(ntargets, "SGA", dtype='>S3'), name='PVTYPE'), 
                     Column(np.full(ntargets, 3, dtype='>i4'), name='PVPRIORITY'),
                     Column(np.full(ntargets, 1, dtype='>i4'), name='POINTINGID'),
                     Column(filtered_data["SGA_ID"], name='SGA_ID')])
target_data.write(savedatapath+"pv_sga.fits", format='fits', overwrite=True)

[False] [False] [ True]
80961 80961


## PV_EXT - Extended objects sample

Our EXT targets are based on galaxies in SGA what are larger than the fibre patrol radius. As such, they are ideal for provide a targetting solution for a fibre that would otherwise have no science target. For these, we provide a grid of targets spaced along the semi-major axis. The following routines are based on Kelly Douglass's code [here](https://github.com/kadglass/DESI_SGA/blob/master/sga_semimajor_grids.ipynb)

The selection function for the EXT sample is:
* inSGA == 1. This means the object is in the SGA catalogue. See function `inSGA_filter` (in previous code block so needs caching)
* Large galaxies with D(26) >= 2.85 arcmins. See function `large_galaxy_filter`

We then restrict our selection to objects with Dec >= -30.0. See function `DESI_footprint_filter`.

We then treat the galaxies differently depending on whether they already made it into the FP or TF samples previously (See functions `FP_shape_filter` and `TF_shape_filter`). If the galaxy is already FP, we add semi-major axis pointings at 0.33, 0.67 and 1.0 times D(26) at lowest priority and with OVERRIDE == TRUE to provide some more targetting solutions if the fibre falls on this galaxy. If the galaxy is already TF, for SV we are doing nothing - these already have semi-major axis pointings at the same factors of D(26) as above. In the main survey, we would add some additional low priority semi-major axis targets with OVERRIDE == TRUE. Finally, if the galaxy is neither FP or TF, we add a central pointing with OVERRIDE == FALSE, and the six semi-major axis pointings with OVERRIDE == TRUE, all again at lowest priority. See function `grid_positions`.

Note that we also provide hand-picked off-axis pointings for these very large galaxies as they could also be very wide, such that semi-major axis pointings alone don't provide enough targets within the SGA ellipse.

**Note, given low redshift success rate from SV3 past 0.4R(26), I upped the number of pointings to include values closer to the centre, while still providing targets at larger radii for targetting solutions. New values are +/-(0.2, 0.4, 0.6, 0.8, 1.0) R(26).**  

In [11]:
def grid_positions(filtered_data, type="other"):
    
    r50 = filtered_data['size_SGA'] / 120.   # arcminutes -> degrees, diameter -> radius
        
    # Centre already included in FP at priority 1, other pointings added at priority 3.
    if type == "fp":
        #x = np.array([-1.0, -0.67, -0.33, 0.33, 0.67, 1.0]).reshape((1,10))
        x = np.array([-1.0, -0.8, -0.6, -0.4, -0.2, 0.2, 0.4, 0.6, 0.8, 1.0]).reshape((1,10))
        priority = np.array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype='>i4')
        place = np.array([True, True, True, True, True, True, True, True, True, True], dtype='bool')
    # Values at 0.4 already included in TF at priority 1, and centre at priority 2. 
    # Let's add other pointings at priority 2 as they might be useful for PV. Note this is unused for SV.
    elif type == "tf":
        #x = np.array([-0.67, -0.33, 0.33, 0.67]).reshape((1,4))
        x = np.array([-1.0, -0.8, -0.6, -0.2, 0.2, 0.6, 0.8, 1.0]).reshape((1,8))
        priority = np.array([2, 2, 2, 2, 2, 2, 2, 2], dtype='>i4')
        place = np.array([True, True, True, True, True, True, True, True], dtype='bool')           
    # Not of interest for FP or TF. Add all pointings at priority 3
    else:
        #x = np.array([-1.0, -0.67, -0.33, 0.0, 0.33, 0.67, 1.0]).reshape((1,7))
        x = np.array([-1.0, -0.8, -0.6, -0.4, -0.2, 0.0, 0.2, 0.4, 0.6, 0.8, 1.0]).reshape((1,11))
        priority = np.array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype='>i4')
        place = np.array([True, True, True, True, True, False, True, True, True, True, True], dtype='bool')     
    
    # Distances along the semi-major axis from the center coordinate for our targets
    delta_a = np.dot(r50.reshape((len(r50),1)),x).T
    
    # Target positions
    og_position = SkyCoord(filtered_data['RA']*u.deg, filtered_data['DEC']*u.deg, frame='icrs')
    fiber_pos = og_position.directional_offset_by(filtered_data['SGA_pa']*u.deg, delta_a*u.deg)
    fiber_ra, fiber_dec = fiber_pos.ra.value.T.flatten(), fiber_pos.dec.value.T.flatten()

    nx = np.shape(x)[1]
    fiber_id = np.tile(np.arange(1, nx+1),len(filtered_data))
    pvpriority = np.tile(priority, len(filtered_data))   # Edges at high prority, centre at low
    override = np.tile(place, len(filtered_data)) # Edges have override, but not centre.
    objid = np.repeat(filtered_data["OBJID"], nx)
    brickid = np.repeat(filtered_data["BRICKID"], nx)
    brickname = np.repeat(filtered_data["BRICKNAME"], nx)
    sga_id = np.repeat(filtered_data["SGA_ID"], nx)

    return fiber_ra, fiber_dec, fiber_id, objid, brickid, brickname, pvpriority, override, sga_id

# Load the DESI tile centres. These are somewhere in NERSC, but not sure where. 
# So grabbed from the wiki. Should update once we work out where they are for posterity.
desi_alltiles = fits.open('desi-tiles.fits', memmap=True)[1].data
desi_tiles = desi_alltiles[desi_alltiles['IN_DESI']==1]

# Read in the north and south properties and concatenate them
fullsweep_north = np.load(savedatapath+'fullsweep_north.npy')
fullsweep_south = np.load(savedatapath+'fullsweep_south.npy')
fullsweep_combi = np.concatenate([fullsweep_north, fullsweep_south],axis=0)

# Only keep objects in the DESI footprint and SGA
SGA_only_filter = inSGA_filter(fullsweep_combi)
footprint_filter = DESI_footprint_filter(fullsweep_combi)#, desi_tiles)
index = np.where(footprint_filter & SGA_only_filter)
fullsweep_combi = fullsweep_combi[index]

# Clean data by removing multiple SGA matches  ..obsolete with DR9
#index = cleanmulti(fullsweep_combi)
#fullsweep_combi = fullsweep_combi[index]

# Keep only galaxies larger than the patrol radius, but which are not in the previous FP or TF selections
# Compute all the FP filters 
mag_filter = FP_mag_filter(fullsweep_combi,fp_rband_limit)
shape_filter_FP = FP_shape_filter(fullsweep_combi,fp_sersic_limit)
colour_filter = FP_colour_filter(fullsweep_combi)
shape_filter_TF = TF_shape_filter(fullsweep_combi,tf_ratiomin,tf_sersic_limit)
large_filter = large_galaxy_filter(fullsweep_combi)
index = np.where(large_filter)
index_FP = np.where(large_filter & shape_filter_FP & mag_filter & colour_filter)
index_TF = np.where(large_filter & shape_filter_TF)
index_other = np.where(large_filter & ~shape_filter_FP & ~shape_filter_TF)

# Output the underlying galaxies 
filtered_data = fullsweep_combi[index]
ngalaxies = len(filtered_data)
#fits.writeto(savedatapath+"pv_ext_full.fits", filtered_data, overwrite=True)

# Get the target positions by perturbing along the semi-major axis. We need to do this differently if the EXT galaxy already
# made it into the FP or TF selections to avoid duplicating targets, and get the correct priorities.
filtered_data_FP = fullsweep_combi[index_FP]
filtered_data_TF = fullsweep_combi[index_TF]
filtered_data_other = fullsweep_combi[index_other]
data_FP = grid_positions(filtered_data_FP, type="fp")
data_TF = grid_positions(filtered_data_TF, type="tf")
data_other = grid_positions(filtered_data_other, type="other")
fiber_ra = np.concatenate([data_FP[0], data_TF[0], data_other[0]])
fiber_dec = np.concatenate([data_FP[1], data_TF[1], data_other[1]])
fiber_id = np.concatenate([data_FP[2], data_TF[2], data_other[2]])
objid = np.concatenate([data_FP[3], data_TF[3], data_other[3]])
brickid = np.concatenate([data_FP[4], data_TF[4], data_other[4]])
brickname = np.concatenate([data_FP[5], data_TF[5], data_other[5]])
pvpriority = np.concatenate([data_FP[6], data_TF[6], data_other[6]])
override = np.concatenate([data_FP[7], data_TF[7], data_other[7]])
sga_id = np.concatenate([data_FP[8], data_TF[8], data_other[8]])

#For SV, we don't need to reinclude the extra TF pointings as they are already there.
"""fiber_ra = np.concatenate([data_FP[0], data_other[0]])
fiber_dec = np.concatenate([data_FP[1], data_other[1]])
fiber_id = np.concatenate([data_FP[2], data_other[2]])
objid = np.concatenate([data_FP[3], data_other[3]])
brickid = np.concatenate([data_FP[4], data_other[4]])
brickname = np.concatenate([data_FP[5], data_other[5]])
pvpriority = np.concatenate([data_FP[6], data_other[6]])
override = np.concatenate([data_FP[7], data_other[7]])"""
ntargets = len(fiber_ra)
print(len(filtered_data_FP), len(filtered_data_TF), len(filtered_data_other), ntargets)    
    
# Output the targets
target_data = Table([Column(objid, name='OBJID'),
                     Column(brickid, name='BRICKID'),
                     Column(brickname, name='BRICKNAME'),
                     Column(fiber_ra, name='RA'), 
                     Column(fiber_dec, name='DEC'), 
                     Column(np.zeros(ntargets, dtype='>f4'), name='PMRA'), 
                     Column(np.zeros(ntargets, dtype='>f4'), name='PMDEC'), 
                     Column(np.full(ntargets, 2015.5, dtype='>f4'), name='REF_EPOCH'), 
                     Column(override, name='OVERRIDE'), 
                     Column(np.full(ntargets, "EXT", dtype='>S3'), name='PVTYPE'), 
                     Column(pvpriority, name='PVPRIORITY'),
                     Column(fiber_id, name='POINTINGID'),
                     Column(sga_id, name='SGA_ID')])
#target_data.write(savedatapath+"pv_ext.fits", format='fits', overwrite=True)

344 590 187 10217


## Reordering into bright and dark time lists and different priorities

We now have lists of FP, TF, SGA and EXT targets from the above codes. We also have a set of hand-picked targets for very large SGA galaxies, denoted as EOA from here on. Given these the last thing to do is recombine them based on their relative priority. The split is based on PV_PRIORITY, and targets are duplicated in bright and dark time:

In [12]:
np.random.seed(2011)

# Read in the pv_fp, pv_tf, pv_sga, pv_ext and pv_eoa files
pv_fp = fits.open(savedatapath+'pv_fp.fits', memmap=True)[1].data
pv_tf = fits.open(savedatapath+'pv_tf.fits', memmap=True)[1].data
pv_sga = fits.open(savedatapath+'pv_sga.fits', memmap=True)[1].data
pv_ext = fits.open(savedatapath+'pv_ext.fits', memmap=True)[1].data
pv_eoa = fits.open(savedatapath+'SGA_off-axis_targets_deccut.fits', memmap=True)[1].data
ntargets = len(pv_fp) + len(pv_tf) + len(pv_sga) + len(pv_ext) + len(pv_eoa)
print(ntargets)

# Randomise these into pv_bright and pv_dark lists. First do the TF targets. Keep galaxies with multiple
# pointings consecutive in case list gets truncated or something.
npointings_tf = np.amax(pv_tf["POINTINGID"])
pv_tf_sortid = npointings_tf * np.random.permutation(len(pv_tf[pv_tf["POINTINGID"] == 1]))
pv_tf_idlist = np.concatenate([range(i,i+npointings_tf) for i in pv_tf_sortid])
pv_tf_randomised = pv_tf[pv_tf_idlist]

# Now the EXT targets. Let's just randomly reorder these as they have different numbers of pointings
# and truncation is less important.
pv_ext_sortid = np.random.permutation(len(pv_ext))
pv_ext_randomised = pv_ext[pv_ext_sortid]

# Now the SGA targets
pv_sga_sortid = np.random.permutation(len(pv_sga))
pv_sga_randomised = pv_sga[pv_sga_sortid]

# Finally the Extended Off-Axis (EOA) targets for very large galaxies. 
# These are just a list of RAs and Decs, so let's create the supplementary information here. They are in SGA, but I (Cullan) 
# doesn't know them, so the SGA IDs have been set to -2
neoa = len(pv_eoa)
pv_eoa_randomised = np.zeros(neoa, dtype=galaxydatatype)
pv_eoa_randomised["RA"] = pv_eoa["RA"]
pv_eoa_randomised["DEC"] = pv_eoa["DEC"]
pv_eoa_randomised["OBJID"] = np.zeros(neoa, dtype='>i8')-1
pv_eoa_randomised["BRICKID"] = np.zeros(neoa, dtype='>i4')-1
pv_eoa_randomised["BRICKNAME"] = np.full(neoa, "None", dtype='S8')
pv_eoa_randomised["PMRA"] = np.zeros(neoa, dtype='>f4')
pv_eoa_randomised["PMDEC"] = np.zeros(neoa, dtype='>f4')
pv_eoa_randomised["REF_EPOCH"] = np.full(neoa, 2015.5, dtype='>f4')
pv_eoa_randomised["OVERRIDE"] = np.full(neoa, True, dtype='bool')
pv_eoa_randomised["PVTYPE"] = np.full(neoa, "EOA", dtype='>S3')
pv_eoa_randomised["PVPRIORITY"] = np.full(neoa, 3, dtype='>i4')
pv_eoa_randomised["POINTINGID"] = np.full(neoa, 1, dtype='>i4')
pv_eoa_randomised["SGA_ID"] = np.full(neoa, -2, dtype='>i8')

# Randomly insert the FP and SGA targets into the TF targets (keeping the npointing blocks together!) and then append the SGA, EXT and EOA targets.
pv_fp_insertid = npointings_tf * np.random.randint(len(pv_tf_randomised[pv_tf_randomised["POINTINGID"] == 1]),size=len(pv_fp))
pv_combined = np.insert(pv_tf_randomised, pv_fp_insertid, pv_fp)

full_data = np.zeros(ntargets, dtype=galaxydatatype)
for name in ["OBJID", "BRICKID", "BRICKNAME", "RA", "DEC", "PMRA", "PMDEC", "REF_EPOCH", "OVERRIDE", "PVTYPE", "PVPRIORITY", "POINTINGID", "SGA_ID"]:
    full_data[name]= np.concatenate([pv_combined[name], pv_ext_randomised[name], pv_sga_randomised[name], pv_eoa_randomised[name]])
print(full_data["PVTYPE"], full_data["OVERRIDE"])
                                 
# Output the targets into different priority lists
for priority, name in enumerate(["HIGH", "MEDIUM", "LOW"]):
    filename_dark = "PV_DARK_" + name + ".fits"
    filename_bright = "PV_BRIGHT_" + name + ".fits"
    priority_index = np.where(full_data["PVPRIORITY"] == priority+1)
    print(len(priority_index[0]))
    pv_data = Table([Column(full_data["OBJID"][priority_index], name='OBJID'),
                         Column(full_data["BRICKID"][priority_index], name='BRICKID'),
                         Column(full_data["BRICKNAME"][priority_index], name='BRICKNAME'),
                         Column(full_data["RA"][priority_index], name='RA'), 
                         Column(full_data["DEC"][priority_index], name='DEC'), 
                         Column(full_data["PMRA"][priority_index], name='PMRA'), 
                         Column(full_data["PMDEC"][priority_index], name='PMDEC'), 
                         Column(full_data["REF_EPOCH"][priority_index], name='REF_EPOCH'), 
                         Column(full_data["OVERRIDE"][priority_index], name='OVERRIDE'), 
                         Column(full_data["PVTYPE"][priority_index], name='PVTYPE'), 
                         Column(full_data["PVPRIORITY"][priority_index], name='PVPRIORITY'),
                         Column(full_data["POINTINGID"][priority_index], name='POINTINGID'),
                         Column(full_data["SGA_ID"][priority_index], name='SGA_ID')])
    pv_data.write(savedatapath+filename_bright, format='fits', overwrite=True)
    pv_data.write(savedatapath+filename_dark, format='fits', overwrite=True)

FileNotFoundError: [Errno 2] No such file or directory: '/global/homes/k/ksaid/DESI_Work/desi_pv/savepath_dr9_corr/pv_ext.fits'